# Experiment 1: Model Validation

### Description

Plot key system variables and validate that the model is working as expected.

### Assumptions

### Experiment Setup

There are several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

In [ ]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook-configuration methods, such as loading IPython modules
import setup

# External dependencies
import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pprint import pprint

import visualizations
from experiments.run import run
from experiments.utils import display_code

In [ ]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = True

In [ ]:
# Import experiment templates
import experiments.templates.time_domain_analysis as time_domain_analysis

In [ ]:
# Inspect experiment template:
#display_code(time_domain_analysis)

In [ ]:
# Create a new copy of the relevant simulation for each analysis
simulation_1 = copy.deepcopy(time_domain_analysis.experiment.simulations[0])

# Adjust experiment

### Override Initial State

In [ ]:
# Check initial state:
pprint(simulation_1.model.initial_state)

In [ ]:
simulation_1.model.initial_state.update({
    "hosts": 1, 
    #"clients": 1,
    #"avg_price": 4
})

### Override parameters

In [ ]:
pprint(simulation_1.model.params)

In [ ]:
simulation_1.model.params.update({
   # "host_setup_delay": [90],
})

In [ ]:
# Execute the simulation (radCAD):

df_1, _exceptions = run(simulation_1)

In [ ]:
# View DataFrame:
# df_1.head(100)

# Anaysis 1: User Adoption

Plot user adoption over the time horizon.

The rate of user adoption at any given timestep is a function of: 
- **word-of-mouth** (the more clients there are, the more likely )
- **base onboarding rate** (set in system parameters as 'onboarding_coeff')
- **price attractiveness** for the given agent (i.e. lower price = more attractive to clients but less attractive for hosts) 
- **registration delay** (i.e the average time clients take to register)

In [ ]:
# fig_df = df_1.query('variable_name == 2')

fig = px.line(
    df_1,
    x='timestep',
    y=['hosts', 'clients', 'potential_users'],
    height=300
)

fig.update_layout(
    title="User Adoption",
    xaxis_title="Days",
    yaxis_title="Persons",
    legend_title="User Type")

fig.show()

# Anaysis 2: Network Allocation

Plot the actual bandwidth allocation for the network over time.

Notes: 
- Since the network assumes clients can initiate temporary connections, actual network allocation is dependent on the current price attractiveness to existing clients, v.s. the 

In [ ]:
fig = px.line(
    df_1,
    x='timestep',
    y=['network_allocation'],
    height=350
)

fig.update_layout(
    title="Network Allocation",
    xaxis_title="Days",
    yaxis_title="Mbps",
    legend_title="Allocation")

fig.show()

In [ ]:
# Use Pandas to query results 

#E.g. maximum network allocation
df_1.groupby('subset')['network_allocation'].max()

In [ ]:
fig = px.line(
    df_1,
    x='timestep',
    y=['avg_price'],
    height=300
)

fig.update_layout(
    title="AVG Host Price over Time",
    xaxis_title="Days",
    yaxis_title="ZAR/Mbps/Day",
    legend_title="Price")

fig.show()


In [ ]:
# Plot 3D Surface:

# visualizations.plot_surface(df_1)

In [ ]:
fig = px.line(
    df_1,
    x='timestep',
    y=['network_penetration'],
    height=300
)

fig.update_layout(
    title="Network Penetration",
    xaxis_title="Days",
    yaxis_title="%",
    legend_title="%")

fig.show()

# Anaysis: Yields

Host and platform yeilds

In [ ]:
fig = px.line(
    df_1,
    x='timestep',
    y=['hosts_daily_revenue', 'hosts_daily_profit', 'platform_daily_revenue'],
    height=400
)

fig.update_layout(
    title="Daily Revenue & Profit Yields",
    xaxis_title="Days",
    yaxis_title="ZAR/Day",
    legend_title="Agents"
)

newnames = {'hosts_daily_revenue':'Hosts (revenue)', 'hosts_daily_profit': 'Hosts (profit)', 'platform_daily_revenue': 'Platform (revenue)'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
        
fig.show()

# Analysis: Hosts & Platform Yields

In [ ]:
# Simulation metrics
period = df_1['timestep'].max()
service_fee = simulation_1.model.params["service_fee"][0]

# Pandas calculations:
host_cumulative_revenue = int(df_1['hosts_daily_revenue'].sum())
host_cumulative_profit = int(df_1['hosts_daily_profit'].sum())
platform_cumulative_revenue = int(df_1['platform_daily_revenue'].sum())

# Output formatting:
service_fee_formatted = service_fee * 100
revenue_formatted = "R{:,.0f}".format(host_cumulative_revenue)
profit_formatted = "R{:,.0f}".format(host_cumulative_profit)
platform_revenue_formatted = "R{:,.0f}".format(platform_cumulative_revenue)

# Display output:
print("Host cumulative revenue (", period , "days ):")
print(revenue_formatted, "\n")

print("Host cumulative profit (", period , "days ):")
print(profit_formatted, "\n")

print("Platform cumulative revenue (", service_fee_formatted, "% fee) (", period , "days ):")
print(platform_revenue_formatted, "\n")